# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from regions import boolean_mask, SOM_area, global_ocean, SOM_mask
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev
from xr_DataArrays import xr_AREA
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, lag_linregress_3D

# global yearly avg. SST map and trends

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' )
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' )

# SST indices

In [ ]:
MASK_ocn  = boolean_mask('ocn'     , 0)
MASK_low  = boolean_mask('ocn_low' , 0)
MASK_rect = boolean_mask('ocn_rect', 0)

In [ ]:
TAREA       = xr_AREA('ocn')
AREA_low    = xr_AREA('ocn_low')
AREA_rect   = xr_AREA('ocn_rect')

# Southern Ocean

## Southern Ocean Mode index
annual SST anomaly (50S-35S, 0E-50W), here detrended

In [ ]:
SOMi_area     = TAREA.where(SOM_mask('ocn')    ).sum()
SOMi_area_low = AREA_low.where(SOM_mask('ocn_low')).sum()
print(SOMi_area, SOMi_area_low)

In [ ]:
%%time
# 30 sec
dims = ('nlat', 'nlon')
SOMi_ctrl = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA, index_loc=None, AREA_index=SOMi_area, MASK=SOM_mask('ocn'), dims=dims)
SOMi_rcp  = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA, index_loc=None, AREA_index=SOMi_area, MASK=SOM_mask('ocn'), dims=dims)
SOMi_lpd  = SST_index(xa_SST=SST_yrly_lpd , AREA=AREA_low, index_loc=None, AREA_index=SOMi_area_low, MASK=SOM_mask('ocn_low'), dims=dims)
SOMi_lpi  = SST_index(xa_SST=SST_yrly_lpi , AREA=AREA_low, index_loc=None, AREA_index=SOMi_area_low, MASK=SOM_mask('ocn_low'), dims=dims)

In [ ]:
f,ax = plt.subplots(1,1,figsize=(12,5), sharex=True)

ax.tick_params(labelsize=14)
ax.set_xlabel('time [years]', fontsize=16)
    
ax.set_ylabel('SOM area temperature [$^\circ$C]', fontsize=16)
L1, = ax.plot(SOMi_ctrl.time/365+1850, SOMi_ctrl, lw=.5, ls=':', c='C0', label='yearly data')
ax.plot(SOMi_rcp .time/365+ 200, SOMi_rcp , lw=.5, ls=':', c='C1')
ax.plot(SOMi_lpd .time/365+1350, SOMi_lpd , lw=.5, ls=':', c='C2')
ax.plot(SOMi_lpi .time/365-1600, SOMi_lpi , lw=.5, ls=':', c='C3')

L2, = ax.plot(SOMi_ctrl.time/365+1850, lowpass(SOMi_ctrl, 13), c='C0', label='lowpass 13 yr')
ax.plot(SOMi_rcp .time/365+ 200, lowpass(SOMi_rcp , 13), c='C1')
ax.plot(SOMi_lpd .time/365+1350, lowpass(SOMi_lpd , 13), c='C2')
ax.plot(SOMi_lpi .time/365-1600, lowpass(SOMi_lpi , 13), c='C3')

L3, = ax.plot(SOMi_ctrl.time/365+1850, xr_lintrend(SOMi_ctrl), c='C0', ls='-.', lw=1.5, label='linear trend')
ax.plot(SOMi_rcp .time/365+ 200, xr_lintrend(SOMi_rcp ), c='C1', ls='-.', lw=1.5)
ax.plot(SOMi_lpd .time/365+1350, xr_lintrend(SOMi_lpd ), c='C2', ls='-.', lw=1.5)
ax.plot(SOMi_lpi .time/365-1600, xr_lintrend(SOMi_lpi ), c='C3', ls='-.', lw=1.5)

ax.legend(handles=[L1,L2,L3], fontsize=14, ncol=3, frameon=False)

ax.text(1950, 12.3, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, 12.3, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, 12.3, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, 12.3, 'pre-ind. low' , fontsize=16, color='C3')

plt.tight_layout()
plt.savefig(f'{path_results}/SST/SOM_overview_raw')

detrended

In [ ]:
SOMi_detr_ctrl = xr_lintrend(SOMi_ctrl)
SOMi_detr_rcp  = xr_lintrend(SOMi_rcp )
SOMi_detr_lpd  = xr_lintrend(SOMi_lpd )
SOMi_detr_lpi  = xr_lintrend(SOMi_lpi )

SOMi_detr_ctrl.to_netcdf(f'{path_results}/SST/SOM_index_ctrl.nc')
SOMi_detr_rcp .to_netcdf(f'{path_results}/SST/SOM_index_rcp.nc' )
SOMi_detr_lpd .to_netcdf(f'{path_results}/SST/SOM_index_lpd.nc' )
SOMi_detr_lpi .to_netcdf(f'{path_results}/SST/SOM_index_lpi.nc' )

In [ ]:
SOMi_detr_ctrl = xr.open_dataarray(f'{path_results}/SST/SOM_index_ctrl.nc', decode_times=False)
SOMi_detr_rcp  = xr.open_dataarray(f'{path_results}/SST/SOM_index_rcp.nc' , decode_times=False)
SOMi_detr_lpd  = xr.open_dataarray(f'{path_results}/SST/SOM_index_lpd.nc' , decode_times=False)
SOMi_detr_lpi  = xr.open_dataarray(f'{path_results}/SST/SOM_index_lpi.nc' , decode_times=False)

In [ ]:
f,ax = plt.subplots(1,1,figsize=(12,5), sharex=True)

ax.tick_params(labelsize=14)
ax.set_xlabel('time [years]', fontsize=16)

ax.text(1950, .3, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .3, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .3, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .3, 'pre-ind. low' , fontsize=16, color='C3')


ax.set_ylabel('detrended SOM index [$^\circ$C]', fontsize=16)
ax.axhline(0, c='k', lw=.5)
ax.plot(SOMi_ctrl.time/365+1850, chebychev(SOMi_detr_ctrl.values, 13), c='C0', ls='--', label='chebychev 13 yr')
ax.plot(SOMi_rcp .time/365+ 200, chebychev(SOMi_detr_rcp .values, 13), c='C1', ls='--')
ax.plot(SOMi_lpd .time/365+1350, chebychev(SOMi_detr_lpd .values, 13), c='C2', ls='--')
ax.plot(SOMi_lpi .time/365-1600, chebychev(SOMi_detr_lpi .values, 13), c='C3', ls='--')

ax.plot(SOMi_ctrl.time/365+1850, lowpass(SOMi_detr_ctrl.values, 40), c='C0', lw=2, label='lowpass 40 yr')
ax.plot(SOMi_rcp .time/365+ 200, lowpass(SOMi_detr_rcp .values, 40), c='C1', lw=2)
ax.plot(SOMi_lpd .time/365+1350, lowpass(SOMi_detr_lpd .values, 40), c='C2', lw=2)
ax.plot(SOMi_lpi .time/365-1600, lowpass(SOMi_detr_lpi .values, 40), c='C3', lw=2)

ax.legend(fontsize=14, ncol=2, frameon=False, loc=4)
ax.set_xticks(np.arange(0,2400,200))

plt.tight_layout()
plt.savefig(f'{path_results}/SST/SOM_overview')

In [ ]:
SOMi_detr_ctrl = xr.open_dataarray(f'{path_results}/SST/SOM_index_ctrl.nc', decode_times=False)
SOMi_detr_rcp  = xr.open_dataarray(f'{path_results}/SST/SOM_index_rcp.nc' , decode_times=False)


In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

# plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp , c='C1', lw=.5)

plt.plot(SOMi_ctrl.time/365     , SOMi_trend_ctrl[1] + SOMi_trend_ctrl[0]*SOMi_ctrl.time, ls='--')
plt.plot(SOMi_rcp .time/365-1800, SOMi_trend_rcp [1] + SOMi_trend_rcp [0]*SOMi_rcp.time , ls='--')

L1, = plt.plot(SOMi_ctrl.time/365     , SOMi_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
L2, = plt.plot(SOMi_rcp .time/365-1800, SOMi_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('raw SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2, loc=4, handles=[L1, L2])
# plt.savefig(f'{path_results}/SST/SOM_raw')

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

plt.axhline(0,c='k', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl, c='C0', lw=.5)
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp , c='C1', lw=.5)
plt.plot(SOMi_ctrl.time/365     , SOMi_detr_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='CTRL')
plt.plot(SOMi_rcp .time/365-1800, SOMi_detr_rcp .rolling({'time':5}, center=True).mean(), c='C1', lw=2, label='RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2)
# plt.savefig(f'{path_results}/SST/SOM_index')

In [ ]:
SOMi_detr_ctrl.plot()
(SOMi_ctrl - xr_lintrend(SOMi_ctrl)).plot()

## spectra

In [ ]:
SOMis = [SOMi_ctrl, SOMi_rcp, SOMi_lpd, SOMi_lpi]

f, ax = plt.subplots(1, 1, figsize=(8,5), sharey=True)
ax.set_ylabel('spectral power', fontsize=16)

for i in range (4):
    label= ['CTRL', 'RCP', 'LPD', 'LPI'][i]
    f, Pxx = sp.signal.welch(SOMis[i]-xr_lintrend(SOMis[i]), fs=1)
    ax.loglog(1/f, np.sqrt(Pxx), label=label)

ax.tick_params(labelsize=14)
ax.set_xlabel('period [yr]', fontsize=16)
ax.legend(fontsize=14, frameon=False)
    
plt.tight_layout()

## regression maps

In [ ]:
SST_yrly_detr_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc', decode_times=False)
SST_yrly_detr_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_rcp.nc' , decode_times=False)
SST_yrly_detr_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_lpd.nc' , decode_times=False)
SST_yrly_detr_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_lpi.nc' , decode_times=False)

In [ ]:
linregr_ctrl = lag_linregress_3D(chebychev(SOMi_detr_ctrl, 13), SST_yrly_detr_ctrl)

In [ ]:
def cor_map(SSTi_detr, SST_yrly_detr, run, index_name):
    """"""
    assert run in ['ctrl', 'rcp', 'lpi', 'lpd']
    
    if index_name=='AMO':
        cntrl_lon = 0
        mask = 
    elif index_name=='SOM':
        cntrl_lon = -30
    elif index_name=='IPO':
        cntrl_lon = 200
        
    # calculate regression maps
    ds = lag_linregress_3D(SSTi_detr, SST_yrly_detr, lagx=0, lagy=0)

    # correlation & statistical significance
    fig = plt.figure(figsize=(8,5))
    ax  = fig.add_subplot(1, 1, 1, projection=ccrs.EqualEarth(central_longitude=cntrl_lon))
    ax.set_position([.02,.05,.96,.93])
    cax, kw = mpl.colorbar.make_axes(ax,location='bottom',pad=0.03,shrink=0.8)
    im = ax.pcolormesh(ds.TLONG,
                       ds.TLAT,
                       ds.cor.values,
                       cmap=discrete_cmap(12, cmocean.cm.balance),
                       vmin=-1, vmax=1,
                       transform=ccrs.PlateCarree() )
    ax.add_feature(cartopy.feature.LAND, zorder=2, edgecolor='black', facecolor='w')
    
    if name in ['AMO', 'SOM']:
        ax.add_patch(mpatches.Polygon(xy=rect_polygon(SST_index_bounds(name)),
                                      facecolor='none',
                                      edgecolor='k',
                                      linewidth=2,
                                      transform=ccrs.PlateCarree()))
    elif name=='IPO':
        for region in ['TPI1', 'TPI2', 'TPI3']:
            ax.add_patch(mpatches.Polygon(xy=rect_polygon(SST_index_bounds(region)),
                                      facecolor='none',
                                      edgecolor='k',
                                      linewidth=2,
                                      transform=ccrs.PlateCarree()))

    ax.text(0, 1, run, ha='left' , va='top', transform=ax.transAxes, fontsize=16)
  
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xlocator = mticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
    
    cbar = fig.colorbar(im, cax=cax, extend='both', **kw)
    cbar.ax.tick_params(labelsize=14)
    label = cbar.set_label('correlation', size=16)
    if filename!=None:  plt.savefig(filename, dpi=100)
        
    return



In [ ]:
cor_map(SSTi_detr, SST_yrly_detr, run='lpd', index_name='AMO')